# Data Inspection 

Before we begin working with the two files, let's get a handle on how they're structured.

## `snp_position.txt`

`snp_position.txt` has 984 lines:

In [16]:
wc -l snp_position.txt

     984 snp_position.txt


There are 15 columns in this file:

In [17]:
head -1 snp_position.txt | awk '{print NF}'

15


This file is 81k big:

In [18]:
ls -lh snp_position.txt | awk '{ print $5 }'

81K


## `fang_et_al_genotypes.txt`

`fang_et_al_genotypes.txt` has 2783 lines:

In [19]:
wc -l fang_et_al_genotypes.txt

    2783 fang_et_al_genotypes.txt


There are 986 columns in this file:

In [20]:
head -1 fang_et_al_genotypes.txt | awk '{print NF}'

986


This file is 11M big:

In [21]:
ls -lh fang_et_al_genotypes.txt | awk '{ print $5 }'

11M


Here is a count of entries per genotype:

In [53]:
cat fang_et_al_genotypes.txt | cut -f 3 | sort | uniq -c

   1 Group
  22 TRIPS
  15 ZDIPL
  17 ZLUXR
  10 ZMHUE
 290 ZMMIL
1256 ZMMLR
  27 ZMMMR
 900 ZMPBA
  41 ZMPIL
  34 ZMPJA
  75 ZMXCH
  69 ZMXCP
   6 ZMXIL
   7 ZMXNO
   4 ZMXNT
   9 ZPERR


# Data Processing



We first begin by making separate directories for maize and teosinte data.  We'll also make an additional directory to keep our raw data in and move the `snp_position.txt` and `fang_et_al_genotypes.txt` into it.

In [26]:
mkdir maize teosinte raw
mv snp_position.txt fang_et_al_genotypes.txt raw

We'll start working with the maize data first.  We begin by extracting all the maize genotype information and saving it to `maize_genotypes.txt`:

In [5]:
cd maize
awk '$3~/ZMMIL|ZMMLR|ZMMMR/' ../raw/fang_et_al_genotypes.txt > maize_genotypes.txt

As a sanity check, we'll see how many genotypes are in `maize_genotypes.txt`:

In [6]:
wc -l maize_genotypes.txt

    1573 maize_genotypes.txt


There are 1573 genotypes, and this agrees with the sum of the number of `ZMMIL`, `ZMMLR`, and `ZMMMR` genotypes in `fang_et_al_genotypes.txt`.  

Before we transpose the `maize_genotypes.txt` in preparation for `join`ing it with `snp_position.txt`, we will prepend the header from `fang_et_al_genotypes.txt` to it.  Unfortunately, this requires the creation of an intermediate file, but `sed` wasn't being a pain in the ass.

In [7]:
head -1 ../raw/fang_et_al_genotypes.txt | cat - maize_genotypes.txt > temp
mv temp maize_genotypes.txt
wc -l maize_genotypes.txt

    1574 maize_genotypes.txt


We then use the provided `transpose.awk` script to transpose the genotype data:

In [8]:
awk -f ../transpose.awk maize_genotypes.txt > transposed_maize_genotypes.txt

We will now merge `transposed_maize_genotypes.txt` and `SNP_ID`.  The merge will be performed on the `Sample_ID` field in `transposed_maize_genotypes.txt` and the `SNP_ID` field in `snp_position.txt`.  As directed in the instructions, bthe merged files first three columns will be `SNP_ID`, `Chromosome`, and `Position`, then all of the remaining columns from `feng_et_al_genotypes.txt`.  

In [9]:
join -o 1.1,1.3,1.4 -o "$(echo 2.{2..1574})" -t $'\t' \
    <(sort -k1 ../raw/snp_position.txt) \
    <(sort -k1 transposed_maize_genotypes.txt) \
    > merged_maize_genotypes.txt

Some explanation:

   * The `-o` argument specifies which columns will be kept in the final merged table.  `1.1` refers to the first column from file 1, `1.2` the second column from file 2, etc.   The second `-o` argument specifies that we want all the remaining columns from file 2, save for the first column (since we already specfied it with `1.1`.
   * The default output delimiter is a space; we switch the delimiting character to tab via the `-t $'\t'` argument so that the data plays nicely later.
   * `join` requires that the two files to be joined be sorted.  We sort the files in place by using the `<(sort ...)` syntax.  
    
We'll now add the header back into the file:

In [10]:
cat <(head -1 ../raw/snp_position.txt | cut -f 1,3,4 | tr '\n' '\t') \
    <(head -1 transposed_maize_genotypes.txt | cut -f 2-1574) | cat - merged_maize_genotypes.txt > temp 
mv temp merged_maize_genotypes.txt

We'll first take a look at `merged_maize_genotypes.txt` to see how many entries there are for each chromosome:

In [11]:
cat merged_maize_genotypes.txt | cut -f 2 | sort | uniq -c

 155 1
  53 10
 127 2
 107 3
  91 4
 122 5
  76 6
  97 7
  62 8
  60 9
   1 Chromosome
   6 multiple
  27 unknown


After we separate each of the entries into separate files by chromosome, we expect there to be 950 total entries for chromosomes 1 through 10, 6 entries with multiple positions, and 27 for unknown positions.  

Before we separate the entries by chromosome, let's make a directory to hold all of the files that will be sorted in increasing order:

In [97]:
mkdir sorted_inc_pos

We can grab all the entries for chromosome 1 and store them in `chromosome_1_sort_inc.txt` by using `awk`:

In [12]:
awk -F '\t' '$2==1' merged_maize_genotypes.txt > ./sorted_inc_pos/chromosome_1_sorted_inc_pos.txt
wc -l ./sorted_inc_pos/chromosome_1_sorted_inc_pos.txt

     155 ./sorted_inc_pos/chromosome_1_sorted_inc_pos.txt


We see that there is 155 entries, matching the the number we found above when `grep`-ing `merged_maize_genotypes.txt`.  

We'll now sort `chromosome_1_sort_inc.txt` on `Position`:

In [13]:
sort -o ./sorted_inc_pos/chromosome_1_sorted_inc_pos.txt \
    -nk3 ./sorted_inc_pos/chromosome_1_sorted_inc_pos.txt

We can visually verify that the sorting is correct:

In [14]:
cat ./sorted_inc_pos/chromosome_1_sorted_inc_pos.txt | cut -f 3 | head -20

157104
3205252
4175293
4175573
4429897
4430055
4835472
4835540
4835596
4835658
4835690
4835949
4836190
4836270
4912321
4912526
8510027
9300391
9300541
10069039


In [15]:
cat ./sorted_inc_pos/chromosome_1_sorted_inc_pos.txt | cut -f 3 | tail -20

286642715
287641039
291738809
291833188
291833767
291833791
291834351
291983425
291983606
291983663
292728854
292728923
293632755
295459549
295771152
298082468
298082504
298082534
298082627
298412984


We'll now replace all missing data values with `?`.  Missing data are encoded by `?/?`.  We'll first count how many missing data there are: 

In [16]:
cat ./sorted_inc_pos/chromosome_1_sorted_inc_pos.txt | grep -c "?/?"

155


We'll now use `sed` to search for all occurrences of `?/?` and replace them with `?`.  Afterwards, we'll verify that there are no more  occurrences of `?/?`:

In [22]:
sed -i '' 's/\?\/\?/\?/g' ./sorted_inc_pos/chromosome_1_sorted_inc_pos.txt
cat ./sorted_inc_pos/chromosome_1_sorted_inc_pos.txt | grep -c "?/?"

0


: 1

So there are no more occurrences of `?/?` in the file.  We also see that there are an equal number of `?` entries in `chromosome_1_sorted_inc_pos.txt`:

In [23]:
cat ./sorted_inc_pos/chromosome_1_sorted_inc_pos.txt | grep -cw "?"

155


We can automate the creation and sorting of the rest of the files:

In [29]:
for i in {1..10}
do
    awk -F '\t' '$2=='$i'' merged_maize_genotypes.txt \
    | sort -nk3 | sed -e 's/\?\/\?/\?/g' \
    > ./sorted_inc_pos/chromosome_${i}_sorted_inc_pos.txt 
done

We verify that the correct number of entries were created:

In [31]:
wc -l ./sorted_inc_pos/chromosome_*

      53 ./sorted_inc_pos/chromosome_10_sorted_inc_pos.txt
     155 ./sorted_inc_pos/chromosome_1_sorted_inc_pos.txt
     127 ./sorted_inc_pos/chromosome_2_sorted_inc_pos.txt
     107 ./sorted_inc_pos/chromosome_3_sorted_inc_pos.txt
      91 ./sorted_inc_pos/chromosome_4_sorted_inc_pos.txt
     122 ./sorted_inc_pos/chromosome_5_sorted_inc_pos.txt
      76 ./sorted_inc_pos/chromosome_6_sorted_inc_pos.txt
      97 ./sorted_inc_pos/chromosome_7_sorted_inc_pos.txt
      62 ./sorted_inc_pos/chromosome_8_sorted_inc_pos.txt
      60 ./sorted_inc_pos/chromosome_9_sorted_inc_pos.txt
     950 total
